# Improving the interpolated area

The previous notebook went well, however only a small region of the target area seems to be interpolated. Let's try and find out why.

In [1]:
from notebook_helpers import *

## Getting set up

You know the drill.

In [2]:
volume_path = Path("resources/d2783eb27223868f57e159f01ce7a1b2.npz")

#
# download_structure_mask("root", Resolution.MICRONS_25)
atlas_array = load_volume(
    get_structure_mask_path("root", Resolution.MICRONS_25), return_raw_array=True
)

aligned_array = np.load(volume_path)["array"]

masked_aligned_array = np.where(atlas_array > 0, aligned_array, 0)

#
# Front-upper-left eighth
focused_area = (
    (0, aligned_array.shape[0] // 2),
    (0, aligned_array.shape[1] // 2),
    (0, aligned_array.shape[2] // 2),
)

#
atlas_array = atlas_array[
    focused_area[0][0] : focused_area[0][1],
    focused_area[1][0] : focused_area[1][1],
    focused_area[2][0] : focused_area[2][1],
]
aligned_array = aligned_array[
    focused_area[0][0] : focused_area[0][1],
    focused_area[1][0] : focused_area[1][1],
    focused_area[2][0] : focused_area[2][1],
]
masked_aligned_array = masked_aligned_array[
    focused_area[0][0] : focused_area[0][1],
    focused_area[1][0] : focused_area[1][1],
    focused_area[2][0] : focused_area[2][1],
]

#
atlas_volume = vedo.Volume(atlas_array)
_ = atlas_volume.cmap(**get_cmap(atlas_volume, vmax=1))
aligned_volume = vedo.Volume(aligned_array)
_ = aligned_volume.cmap(**get_cmap(aligned_volume))
masked_aligned_volume = vedo.Volume(masked_aligned_array)
_ = masked_aligned_volume.cmap(**get_cmap(masked_aligned_volume))

In [3]:
known_coordinates = np.nonzero(masked_aligned_array)
known_points = np.array(known_coordinates).T

known_values = masked_aligned_array[known_coordinates]

target_coordinates = np.nonzero(atlas_array)
target_points = np.array(target_coordinates).T

Quick overview of the problem.

In [4]:
show(
    {
        "Atlas": atlas_volume,
        "Aligned": aligned_volume,
        "Masked": masked_aligned_volume,
    },
    n=3,
)

Quick sanity check

In [5]:
target_array = np.zeros_like(atlas_array)
target_array[target_coordinates] = 1

target_volume = vedo.Volume(target_array)
_ = target_volume.cmap(**get_cmap(target_volume, vmax=1))

In [6]:
show(
    {
        "Atlas": atlas_volume,
        "Target": target_volume,
    },
    n=2,
)

Right, seems the bug was right there. I was halving twice, once when getting the focused area and again when getting the target coordinates.

## For good this time?

In [7]:
neighbours = 3**3

interpolator = RBFInterpolator(
    known_points, known_values, kernel="linear", neighbors=neighbours
)

da_target_points = da.from_array(target_points, chunks=(1_000, 3))
target_values = da.map_blocks(
    partial(interpolation_function, interpolator=interpolator),
    da_target_points,
    drop_axis=1,
    dtype=np.float64,
)

In [8]:
targets_values = target_values.compute()

In [10]:
targets_values.shape

(3135662,)

In [13]:
os.makedirs("resources", exist_ok=True)
np.savez_compressed(
    f"resources/{volume_path.stem}.{neighbours}.interp.npz", array=targets_values
)

## Let's see

In [14]:
interpolated_array = np.zeros_like(masked_aligned_array)
interpolated_array[target_coordinates] = target_values

interpolated_volume = vedo.Volume(interpolated_array)
_ = interpolated_volume.cmap(**get_cmap(interpolated_volume))

In [16]:
np.savez_compressed(
    f"resources/{volume_path.stem}.{neighbours}.interp.array.npz",
    array=interpolated_array,
)

In [15]:
show(
    {
        "Atlas": atlas_volume,
        "Masked": masked_aligned_volume,
        "Interpolated": interpolated_volume,
    },
    n=3,
)